<a href="https://colab.research.google.com/github/RiktikaTalukder/SkillMorph/blob/main/Assignment_5_Data_Preprocessing_and_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
#Load the Data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

from google.colab import drive
drive.mount('/content/drive')

# TODO: Load the housing data
df = pd.read_csv('/content/drive/MyDrive/Dataset/Housing.csv')

# TODO: How many houses are there?
print(f"Number of houses:",len(df))  # Fill this

# TODO: Show first 3 houses
print("First 3 houses:")
display(df.head(3))
# Your code here

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of houses: 545
First 3 houses:


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished


In [49]:
# Look at Columns
# TODO: Show all column names
print("Columns:",list(df.columns))
# Your code here

# TODO: Show data types of columns
print("Data types:")
print(df.dtypes)

Columns: ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']
Data types:
price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object


In [50]:
#Convert Yes/No to 1/0
# Check what's in our data
print("Current mainroad values:", df['mainroad'].head())
# Shows: yes, no, yes, yes, no

# Models need numbers, not text!
# We need: yes → 1, no → 0

label_encoder = LabelEncoder()
binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
for col in binary_cols:
    df[col + '_encoded'] = label_encoder.fit_transform(df[col])


print("After label encoding binary features:")
display(df[[col + '_encoded' for col in binary_cols]].head())

df = df.drop(columns=binary_cols)





Current mainroad values: 0    yes
1    yes
2    yes
3    yes
4    yes
Name: mainroad, dtype: object
After label encoding binary features:


,mainroad_encoded,guestroom_encoded,basement_encoded,hotwaterheating_encoded,airconditioning_encoded,prefarea_encoded
0,1,0,0,0,1,1
1,1,0,0,0,1,0
2,1,0,1,0,0,1
3,1,0,1,0,1,1
4,1,1,1,0,1,0


In [51]:
# Split into Train and Test
# TODO: Split - 80% train, 20% test
X = df.drop('price', axis = 1)
Y = df['price']

furnishingstatus_encoded = pd.get_dummies(df['furnishingstatus'])
X = pd.get_dummies(X, columns=['furnishingstatus'], drop_first=True)

X_train, X_test, Y_train, Y_test = train_test_split(
    X,Y, test_size = 0.2, random_state = 20
    )
print("We will train with:",len(X_train),'samples')
print("We will test with:",len(X_test),'samples')


We will train with: 436 samples
We will test with: 109 samples


In [52]:
#Apply StandardScaler
# Area is huge (1650-13300), Bedrooms is small (1-6)!
#Your Code here
scaler = StandardScaler()
numeric_cols = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])
print("After scaling:")
print(X_train[numeric_cols][:5])

After scaling:
         area  bedrooms  bathrooms   stories   parking
370 -0.400969 -1.333921  -0.562749 -0.920575  1.556577
438 -0.300229 -1.333921  -0.562749 -0.920575 -0.783656
528 -0.542921 -2.712095  -0.562749 -0.920575 -0.783656
119  0.853700  0.044253  -0.562749 -0.920575  1.556577
7    5.057298  2.800602   3.362989  0.259927 -0.783656


In [53]:
#train and test model
#your code here
model = LinearRegression()
model.fit(X_train, Y_train)

print("Model trained!")

Model trained!


In [54]:
# Predict a House Price
# New house details:
# area=5000, bedrooms=3, bathrooms=2, stories=2,
# mainroad=yes, guestroom=no, basement=yes,
# hotwaterheating=no, airconditioning=yes,
# parking=2, prefarea=yes, furnishingstatus=furnished
#your code here
new_house = {
    "area": 5000,
    "bedrooms": 3,
    "bathrooms": 2,
    "stories": 2,
    "parking": 2,
    "mainroad_encoded": 1,
    "guestroom_encoded": 0,
    "basement_encoded": 1,
    "hotwaterheating_encoded": 0,
    "airconditioning_encoded": 1,
    "prefarea_encoded": 1,
    "furnishingstatus_semi-furnished": 0,
    "furnishingstatus_unfurnished": 0
}


new_house_df = pd.DataFrame([new_house])


numeric_cols = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
new_house_df[numeric_cols] = scaler.transform(new_house_df[numeric_cols])


for col in X_train.columns:
    if col not in new_house_df.columns:
        new_house_df[col] = 0


new_house_df = new_house_df[X_train.columns]


predicted_price = model.predict(new_house_df)
print("Predicted house price:", predicted_price[0])


Predicted house price: 7262684.607641464
